### Exercise 7 - Association Rules

First name: Brian
<br>
Last name: Schweigler
<br>
Matriculation number: 16-102-071

#### (1) Take the titanic dataset and using all attributes to predict the class `Survived' (convert age and fare into classes ; exclude names from the attribute list)


#### (a) Choose Three classifiers and evaluate their performance using all attributes;


We will use the same classifiers as in last series, Decision Tree, KNN, Naive Bayes

In [31]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
from mlxtend.evaluate import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE, SelectKBest, f_classif
from sklearn import svm
from mlxtend.frequent_patterns import apriori, association_rules

df = pd.read_csv("data/restaurant.csv")
pd.set_option('display.max_colwidth', None)
le = preprocessing.LabelEncoder()
x_train = df[["choice", "bar", "day", "hungry", "patron", "price", "rain", "booking", "type"]]
x_train = pd.DataFrame(columns=x_train.columns, data=le.fit_transform(x_train.values.flatten()).reshape(x_train.shape))
x_train["time"] = df["time"]

y = le.fit(df["wait"])
y = le.transform(df["wait"])

Xd_train, Xd_test, y_train, y_test = train_test_split(x_train, y, test_size=0.4)

print(Xd_train)
print("y_train = ", y_train, "\n")

print(Xd_test)
print("y_test = ", y_test)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
    choice  bar  day  hungry  patron  price  rain  booking  type  time
2        4    4    3       3       9      0     3        3    10     0
10       3    3    3       3       8      0     3        3    11     0
9        4    4    4       4       6      2     3        4     7    20
5        3    4    3       4       9      1     4        3     7     0
3        4    3    4       4       6      0     3        3    11    20
8        3    4    4       3       6      0     4        3    10    60
11       4    4    4       4       6      0     3        3    10    40
y_train =  [1 0 0 1 1 0 1] 

   choice  bar  day  hungry  patron  price  rain  booking  type  time
7       3    3    3       4       9      1     4        4    11     0
6       3    4    3       3       8      0     4        3    10    20
4       4    3    4       3       6      2     3        4     5    60
0       4    3    3       4       9

In [26]:
# Create a svm classifier using one kernel (linear, polynomial, and radial basis)
clf = svm.SVC(kernel='linear')
clf.fit(Xd_train, y_train)

y_pred = clf.predict(Xd_test)
NB_Accuracy = accuracy_score(y_test, y_pred)

print("y_test with linear kernel = ", y_test)
print("y_pred with linear kernel = ", y_pred, "\n")
print("NB_Accuracy with linear kernel = ", NB_Accuracy, "\n")
print("Confusion Matrix with linear kernel \n", confusion_matrix(y_test, y_pred))

y_test with linear kernel =  [1 0 0 0 1]
y_pred with linear kernel =  [0 1 1 1 1] 

NB_Accuracy with linear kernel =  0.2 

Confusion Matrix with linear kernel 
 [[0 3]
 [1 1]]


In [27]:
clf = svm.SVC(kernel='rbf')
clf.fit(Xd_train, y_train)

y_pred = clf.predict(Xd_test)
NB_Accuracy = accuracy_score(y_test, y_pred)

print("y_test with rbf kernel = ", y_test)
print("y_pred with rbf kernel  = ", y_pred, "\n")
print("NB_Accuracy with rbf kernel  = ", NB_Accuracy, "\n")
print("Confusion Matrix with rbf kernel \n", confusion_matrix(y_test, y_pred))

y_test with rbf kernel =  [1 0 0 0 1]
y_pred with rbf kernel  =  [1 1 1 1 1] 

NB_Accuracy with rbf kernel  =  0.4 

Confusion Matrix with rbf kernel 
 [[0 3]
 [0 2]]


In [28]:
clf = svm.SVC(kernel='poly')
clf.fit(Xd_train, y_train)

y_pred = clf.predict(Xd_test)
NB_Accuracy = accuracy_score(y_test, y_pred)

print("y_test with poly kernel = ", y_test)
print("y_pred with poly kernel = ", y_pred, "\n")
print("NB_Accuracy with poly kernel = ", NB_Accuracy, "\n")
print("Confusion Matrix with poly kernel \n", confusion_matrix(y_test, y_pred))

y_test with poly kernel =  [1 0 0 0 1]
y_pred with poly kernel =  [1 1 1 1 1] 

NB_Accuracy with poly kernel =  0.4 

Confusion Matrix with poly kernel 
 [[0 3]
 [0 2]]


#### Co-occurence matrix

In [42]:
cooc_mat = x_train.T.dot(x_train)
cooc_mat

,choice,bar,day,hungry,patron,price,rain,booking,type,time
choice,157,150,148,155,312,30,141,144,383,960
bar,150,150,144,150,308,27,141,139,378,920
day,148,144,143,147,294,28,136,137,367,980
hungry,155,150,147,157,315,30,143,144,386,900
patron,312,308,294,315,668,60,296,294,789,1600
price,30,27,28,30,60,14,26,31,52,160
rain,141,141,136,143,296,26,136,133,362,860
booking,144,139,137,144,294,31,133,136,352,860
type,383,378,367,386,789,52,362,352,1032,2300
time,960,920,980,900,1600,160,860,860,2300,11600


In [ ]:

# Source: https://www.geeksforgeeks.org/implementing-apriori-algorithm-in-python/#:~:text=Apriori%20Algorithm%20is%20a%20Machine,present%20in%20the%20user's%20cart.
frq_items = apriori(df[["choice", "bar", "day", "hungry", "patron", "price", "rain", "booking", "type"]], min_support = 0.01, use_colnames = True)
# rules = association_rules(frq_items, metric ="support")
# rules = association_rules(frq_items, metric ="confidence")
# rules = association_rules(frq_items, metric ="completeness")
rules = association_rules(frq_items, metric ="lift")
# rules = association_rules(frq_items, metric ="leverage")
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])
print(rules.head())

In [45]:
# Source: https://www.kaggle.com/code/sangwookchn/association-rule-learning-with-scikit-learn
from apyori import apriori
rules = apriori(x_train, min_support = 0.003, min_confidence = 0.2, min_lift = 3, min_length = 2)
results = list(rules)
results = pd.DataFrame(results)
results.head(5)

,items,support,ordered_statistics
0,"(b, k)",0.1,"[((b), (k), 0.5, 5.0), ((k), (b), 1.0, 5.0)]"
1,"(d, y)",0.1,"[((d), (y), 1.0, 3.3333333333333335), ((y), (d), 0.33333333333333337, 3.3333333333333335)]"
2,"(k, g)",0.1,"[((g), (k), 0.5, 5.0), ((k), (g), 1.0, 5.0)]"
3,"(u, g)",0.1,"[((g), (u), 0.5, 5.0), ((u), (g), 1.0, 5.0)]"
4,"(u, h)",0.1,"[((h), (u), 0.5, 5.0), ((u), (h), 1.0, 5.0)]"


#### Explain these measures

Support: The fraction of the total number of data entries in which the variable occurs.

Confidence: Confidence is the conditional probability of occurrence of consequent given the antecedent.

Completeness: TODO

Lift: Controls for the support (frequency) of consequent while calculating the conditional probability of occurrence of Y given X.

Leverage: TODO


